# Generate Model Output from Prompt

This notebook demonstrates how to:
1. Load a language model with the appropriate prompt template
2. Format a user prompt using model-specific templates
3. Generate output from the model

The prompt templates are defined in `func_from_evil_twins.py`

In [1]:
import torch
from func_from_evil_twins import (
    load_model_tokenizer,
    build_prompt,
    PROMPT_TEMPLATES,
    MODEL_NAME_OR_PATH_TO_NAME
)

## Configuration

Set your model name and prompt here. Supported models include:
- `teknium/OpenHermes-2.5-Mistral-7B`
- `meta-llama/Meta-Llama-3-8B-Instruct`
- `mistralai/Mistral-7B-Instruct-v0.2`
- `google/gemma-2-2b-it`
- `HuggingFaceTB/SmolLM2-1.7B-Instruct`
- And many more (see MODEL_NAME_OR_PATH_TO_NAME in func_from_evil_twins.py)

In [2]:
# Configuration
MODEL_NAME = "EleutherAI/pythia-410m"  # Change this to your desired model
USER_PROMPT = " chemical artillery\"?"  # Change this to your prompt
MAX_NEW_TOKENS = 100  # Maximum number of tokens to generate
TEMPERATURE = 1.0  # Sampling temperature (1.0 = default, lower = more deterministic)
TOP_P = 1.0  # Nucleus sampling parameter
USE_FLASH_ATTN_2 = False  # Set to True if you have flash-attention installed

## Load Model and Tokenizer

In [3]:
print(f"Loading model: {MODEL_NAME}")
print(f"Using Flash Attention 2: {USE_FLASH_ATTN_2}")
print()

model, tokenizer = load_model_tokenizer(
    MODEL_NAME,
    dtype=torch.bfloat16,
    device_map="auto",
    use_flash_attn_2=USE_FLASH_ATTN_2,
    eval_mode=True
)

print(f"Model loaded successfully!")
print(f"Model device: {model.device}")
print(f"Model dtype: {model.dtype}")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading model: EleutherAI/pythia-410m
Using Flash Attention 2: False

Model loaded successfully!
Model device: cuda:0
Model dtype: torch.bfloat16


## Show Prompt Template

Display the template being used for this model

In [4]:
# Get the model template name
if MODEL_NAME in MODEL_NAME_OR_PATH_TO_NAME:
    template_name = MODEL_NAME_OR_PATH_TO_NAME[MODEL_NAME]
else:
    # Try to match partial name
    template_name = "default"
    for key in MODEL_NAME_OR_PATH_TO_NAME:
        if key.split("/")[-1] in MODEL_NAME:
            template_name = MODEL_NAME_OR_PATH_TO_NAME[key]
            break

print(f"Using template: {template_name}")
print(f"\nTemplate structure:")
print(f"Prefix: {repr(PROMPT_TEMPLATES[template_name]['prefix'])}")
print(f"Suffix: {repr(PROMPT_TEMPLATES[template_name]['suffix'])}")
print(f"\nFull prompt format:")
print(f"{PROMPT_TEMPLATES[template_name]['prefix']}<YOUR_PROMPT>{PROMPT_TEMPLATES[template_name]['suffix']}")

Using template: pythia

Template structure:
Prefix: ''
Suffix: ''

Full prompt format:
<YOUR_PROMPT>


## Build Formatted Prompt

In [5]:
print(f"User prompt: {USER_PROMPT}")
print()

# Build the prompt with template
prompt_ids, prompt_slice = build_prompt(
    model_name=MODEL_NAME,
    prompt=USER_PROMPT,
    tokenizer=tokenizer,
    validate_prompt=True
)

# Move to model device
prompt_ids = prompt_ids.to(model.device)

print(f"Formatted prompt (full):")
print(tokenizer.decode(prompt_ids[0], skip_special_tokens=False))
print()
print(f"Prompt shape: {prompt_ids.shape}")
print(f"Prompt slice: {prompt_slice}")
print(f"User prompt extracted: {tokenizer.decode(prompt_ids[0, prompt_slice])}")

User prompt:  chemical artillery"?

Formatted prompt (full):
 chemical artillery"?

Prompt shape: torch.Size([1, 3])
Prompt slice: slice(0, 3, None)
User prompt extracted:  chemical artillery"?


## Generate Output

In [6]:
print(f"Generating with parameters:")
print(f"  max_new_tokens: {MAX_NEW_TOKENS}")
print(f"  temperature: {TEMPERATURE}")
print(f"  top_p: {TOP_P}")
print()

# Generate
with torch.no_grad():
    output_ids = model.generate(
        prompt_ids,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=True,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        pad_token_id=tokenizer.eos_token_id,
    )

# Decode the full output
full_output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print("="*80)
print("FULL OUTPUT (with template):")
print("="*80)
print(full_output)
print()

# Extract just the model's response (everything after the prompt)
response_ids = output_ids[0, prompt_ids.shape[1]:]
response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
print("="*80)
print("MODEL RESPONSE ONLY:")
print("="*80)
print(response_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating with parameters:
  max_new_tokens: 100
  temperature: 1.0
  top_p: 1.0

FULL OUTPUT (with template):
 chemical artillery"?
    [4] "What can be done by the Jews?"
    [5] "Where the rabble?"


_Chapter VI._

    One of the great features of the Jew's life is his
    constant self-assertion and the continual war of
    individuality. The Jew is more than a man, there are
    no questions left to say, and a multitude of men may
    indeed not be of the same origin. It is true

MODEL RESPONSE ONLY:

    [4] "What can be done by the Jews?"
    [5] "Where the rabble?"


_Chapter VI._

    One of the great features of the Jew's life is his
    constant self-assertion and the continual war of
    individuality. The Jew is more than a man, there are
    no questions left to say, and a multitude of men may
    indeed not be of the same origin. It is true


## Generate Multiple Samples

Generate multiple responses to see variation

In [7]:
NUM_SAMPLES = 3

print(f"Generating {NUM_SAMPLES} samples...\n")

for i in range(NUM_SAMPLES):
    print(f"{'='*80}")
    print(f"SAMPLE {i+1}:")
    print(f"{'='*80}")
    
    with torch.no_grad():
        output_ids = model.generate(
            prompt_ids,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response_ids = output_ids[0, prompt_ids.shape[1]:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    print(response_text)
    print()

Generating 3 samples...

SAMPLE 1:

I mean the
practical idea of using the cannon.
The reality of a
cannons is, is, you know one can be
of significant impact upon
a vehicle, given that there
are multiple firing
options, or there are certain
options that no, no one can
really determine in advance
and what constitutes a good
impact.
So, like,
a lot of times
we will make a decision here,
we will decide based


SAMPLE 2:


I should also remark that most of my conversations with Russians are via emails. Some even have been over the internet.

My first exchange took place in my apartment on the 16th and was posted in a comment at an e mail-in discussion on the Russian side. The topic was how to get your visa in the US. One of the commentators argued (arguably with truth) that you'd need a $2,500 credit card, but I'm now convinced that my

SAMPLE 3:
 It would seem that this is a clear use of the word warfare that I am not aware of that is not part of our society.
And I was wondering if the ph